In [ ]:
import pandas as pd

"""
Importante:
1 - Classificadores "multilabeled" (podem ter mais de uma label por recurso)
são diferentes de "multiclass"! (Assumem apenas um de mais de 2 rótulos)
2 - Modelos de regressão são utilizados para variáveis contínuas,
para variáveis não contínuas usam-se os classificadores.
Sequência de execução:
1 - Verificar se o dataset é balanceado
2 - Preparação do dataset
3 - Train/test split
"""

df = pd.read_csv('eclipse_jdt.csv')
print(df.columns)
df[['Issue_id', 'Priority', 'Component', 'Title', 'Description']].sample(2) # Pega duas amostras (linhas) aleatórias

print(df.sample(1).T) # T = transpose

df['Priority'].value_counts().sort_index().plot(kind='bar') # Verificação do balanceamento dos rótulos

In [ ]:
df['Component'].value_counts() # Verificação da distribuição dos valores de "component"

In [ ]:
# dropna => Vai retirar os valores vazios da tabela

df = df[['Title','Description','Priority']]
df = df.dropna()

# O título e a descrição serão unidos em uma coluna "text"
df['text'] = df['Title'] + ' ' + df['Description']

# Remove as colunas 'Title' e 'Description', mantendo 'Priority' e 'text'
df = df.drop(columns=['Title','Description'])
df.columns

In [12]:
# Realiza o pré-processamento do corpo do texto
import html
import re

def clean(text):
    # Converte os escapes html em string
    text = html.unescape(text)
    # Remove tags como <tab>
    text = re.sub(r'<[^<>]*>', ' ', text)
    # Remove URL de remarcação [Algum texto](https://....)
    text = re.sub(r'\[([^\[\]]*)\]\([^\(\)]*\)', r'\1', text)
    # Texto ou código entre colchetes como [0]
    text = re.sub(r'\[[^\[\]]*\]', ' ', text)
    # Sequências de caracteres sozinhos, encontra &# mas não #cool
    text = re.sub(r'(?:^|\s)[&#<>{}\[\]+|\\:-]{1,}(?:\s|$)', ' ', text)
    # Sequências de hífens sozinhos como --- or ==
    text = re.sub(r'(?:^|\s)[\-=\+]{2,}(?:\s|$)', ' ', text)
    # Sequências de espaços em branco
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# Retorna apenas os recursos com mais de 50 caracteres
df['text'] = df['text'].apply(clean)
df = df[df['text'].str.len() > 50]
df.sample(2)

,Priority,text
41456,P3,Javadoc does not render properly with WebKitGT...
5677,P3,Synthetic access methods are not reported to b...


In [13]:
# Realiza a separação dos recursos em treino e teste
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(df['text'],
                                                    df['Priority'],
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=df['Priority'])

print('Tamanho dos dados de treino: ', X_train.shape[0])
print('Tamanho dos dados de teste: ', X_test.shape[0])

Tamanho dos dados de treino:  36040
Tamanho dos dados de teste:  9011


In [14]:
# Realiza o processo de vetorização, para posteriormente alimentar o classificador
from sklearn.feature_extraction.text import TfidfVectorizer

# O TF-IDF retorna uma matriz esparsa. O classificador LinearSVC funciona melhor desta forma.
tfidf = TfidfVectorizer(min_df = 10, ngram_range=(1,2), stop_words="english")
X_train_tf = tfidf.fit_transform(X_train)

In [15]:
# Treinando o classificador Suport Vector 
from sklearn.svm import LinearSVC

model1 = LinearSVC(random_state=0, tol=1e-5)
model1.fit(X_train_tf, Y_train)

# Outros parâmetros do classificador
"""LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=0, tol=1e-05,
          verbose=0)"""

LinearSVC(random_state=0, tol=1e-05)

In [ ]:
# Avaliação do modelo utilizado